In [ ]:
pip --version

In [ ]:
%%capture
pip install prometheus-api-client

In [ ]:
%%capture
pip install matplotlib

In [ ]:
from prometheus_api_client import *
import matplotlib.pyplot as plt
import os
import numpy as np
import sys
import datetime
import pandas
import seaborn as sns
# Set default Seaborn style
sns.set(style="darkgrid")
sns.set_context("talk", font_scale=1.4)
## hides ipython warnings
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (30, 10)
#plt.rcParams['figure.figsize'] = (20, 5)

# Extracting data out of ACM Observability for Offline Analysis :

## To set up

1. Install ACM and Observability
1. Get URL of the route called rbac-query-proxy created in namespace:open-cluster-management-observability
1. Get the bearer token of the logged in user
1. Select the time range in the cell below

## If you run this notebook
1. It will create a dataframe in line with the PromQL query and timestamp set
1. And dump the data in a csv in the local file system. 
1. The user will be rbac-ed as per their roles set up in ACM

## Note
1. We run the analysis of this data in a separate notebook
1. However there is nothing stopping us from the doing this analysis inline without saving data in a csv. 
1. A real data set produced from a run is stored in `data/alert-2.csv`. You can use this data and analyze following instructions on another notebook.

### Sets the connection details

In [ ]:
# URL of the route call rbac-query-proxy 
# created in namespace:open-cluster-management-observability
# after installing Observability on Advanaced Cluster Managment Install
url = "https://rbac-query-proxy-open-cluster-management-observability.apps.xyz.redhat.com/"
# bearer token obtained from command `oc whoami -t`
token = "sha256~5AyxhxNb4voPuT0galw4X-abcd1234xyz"
#connects to Thanos or Prometheus as dictated by the URL
pc = PrometheusConnect(url=url, headers={"Authorization": "Bearer {}".format(token)}, disable_ssl=True);

### Sets the time range and time step for all queries below

In [ ]:
start_time=(datetime.datetime.now() - datetime.timedelta(minutes=2160))
end_time=datetime.datetime.now()
#interval between data points gathered
step='10m'

### Runs Query and Saves data

In [ ]:
# PromQL being run: count(ALERTS{alertstate="firing"}) by (cluster,alertname)
alert_total = pc.custom_query_range(
    query='count(ALERTS{alertstate="firing"}) by (cluster,alertname)',
    start_time=start_time,
    end_time=end_time,
    step=step,
)

alert_total_df = MetricRangeDataFrame(alert_total);
alert_total_df["value"]=alert_total_df["value"].astype(float)
alert_total_df.index= pandas.to_datetime(alert_total_df.index, unit="s")

print(alert_total_df.head())
print(alert_total_df.count())

# Saves the data in path /Users/jbanerje/code/multicluster-notebooks/data/alert-2.csv
#alert_total_df.to_csv (r'/Users/jbanerje/code/multicluster-notebooks/data/alert-2.csv', index = True, header=True)


